# Connect to CockroachDB Cloud using pyservicebinding

## Install prerequisite libraries for Service Binding and PostgreSQL (CockroachDB) API access.

In [14]:
!pip install pyservicebinding
!pip install psycopg2-binary

## Check if the Service Binding has been setup.

In [7]:
import os
import pyservicebinding
from pyservicebinding import binding

if 'SERVICE_BINDING_ROOT' in os.environ:
    print('DB binding ok')
else:
    print('DB binding missing')

DB binding ok


## Using pyservicebinding library get credentials for CockroachDB Cloud.

In [8]:
sb = binding.ServiceBinding()
bindings_list = sb.bindings('postgresql', 'CockroachDB Cloud')
if (len(bindings_list)) > 0:
    print('CockroachDB instance - found')
else:
    print('CockroachDB instance - not found')

CockroachDB instance - found


## Use the credentials from the service binding to connect securely (SSL) to CockroachDB Cloud.

In [9]:
import psycopg2
import psycopg2.errorcodes
with open('./root.crt', 'w') as f:
    f.write(bindings_list[0].get('root.crt'))
db_connection = psycopg2.connect(database=bindings_list[0].get('database'), \
    user=bindings_list[0].get('username'), \
    password=bindings_list[0].get('password'), \
    sslmode=bindings_list[0].get('sslmode'), \
    sslrootcert='./root.crt', \
    host=bindings_list[0].get('host'), \
    options=bindings_list[0].get('options'), \
    port=bindings_list[0].get('port'))
print(db_connection)

<connection object at 0x7f27c192d900; dsn: 'user=kubeflow_user.a_new_test_cluster_6e2d20b40c password=xxx dbname=defaultdb host=free-tier14.aws-us-east-1.cockroachlabs.cloud port=26257 options=--cluster=a-new-test-cluster-1075 sslmode=verify-full sslrootcert=./root.crt', closed: 0>


## On the connected Postgres database, run a SQL query on the **accounts** table <br>and ingest the results into a Pandas dataframe for processing the data.

In [15]:
import pandas as pd

cursor = db_connection.cursor()
cursor.execute("SELECT id, balance FROM accounts")
df = pd.DataFrame(cursor.fetchall(), columns = ['ID', 'Amount'])
df.head()

,ID,Amount
0,1,900
1,2,350
